In [1]:
import descartes, matplotlib.pyplot as plt 
import fiona
import geopandas as gpd
import pandas as pd
import censusdata
import gc
gc.collect()

0

In [2]:
fp = "/Users/kmbrgandhi/Documents/Housing/Property_Assessments/propertyassessments/ACS2018/tl_2018_25_bg/tl_2018_25_bg.shp"

In [3]:
acs2018sf = gpd.read_file(fp)

In [4]:
censusdata.printtable(censusdata.censustable('acs5', 2018, 'B15003'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B15003_001E  | EDUCATIONAL ATTAINMENT FOR THE | !! Estimate Total                                        | int  
B15003_002E  | EDUCATIONAL ATTAINMENT FOR THE | !! !! Estimate Total No schooling completed              | int  
B15003_003E  | EDUCATIONAL ATTAINMENT FOR THE | !! !! Estimate Total Nursery school                      | int  
B15003_004E  | EDUCATIONAL ATTAINMENT FOR THE | !! !! Estimate Total Kindergarten                        | int  
B15003_005E  | EDUCATIONAL ATTAINMENT FOR THE | !! !! Estimate Total 1st grade                           | int  
B15003_006E  | EDUCATIONAL ATTAINMENT FOR THE | !! !! Estimate Total 2nd grade                           | int  
B15003_007E  | EDUCATIONAL ATTAINMENT FOR THE | !! !! Estimate Total 3rd grade               

In [5]:
counties = censusdata.geographies(censusdata.censusgeo([('state', '25'), ('county', '*')]), 'acs5', 2018)
counties

{'Suffolk County, Massachusetts': censusgeo((('state', '25'), ('county', '025'))),
 'Barnstable County, Massachusetts': censusgeo((('state', '25'), ('county', '001'))),
 'Dukes County, Massachusetts': censusgeo((('state', '25'), ('county', '007'))),
 'Middlesex County, Massachusetts': censusgeo((('state', '25'), ('county', '017'))),
 'Berkshire County, Massachusetts': censusgeo((('state', '25'), ('county', '003'))),
 'Essex County, Massachusetts': censusgeo((('state', '25'), ('county', '009'))),
 'Bristol County, Massachusetts': censusgeo((('state', '25'), ('county', '005'))),
 'Hampden County, Massachusetts': censusgeo((('state', '25'), ('county', '013'))),
 'Plymouth County, Massachusetts': censusgeo((('state', '25'), ('county', '023'))),
 'Franklin County, Massachusetts': censusgeo((('state', '25'), ('county', '011'))),
 'Norfolk County, Massachusetts': censusgeo((('state', '25'), ('county', '021'))),
 'Hampshire County, Massachusetts': censusgeo((('state', '25'), ('county', '015'))

In [6]:
list_of_counties = []
county_data = {}
for county in counties:
    county_name = county[:-15]
    print(county_name)
    county_val = counties[county]
    county_number = county_val.geo[1][1]
    county_data[county_name] = censusdata.download('acs5', 2018,
                             censusdata.censusgeo([('state', '25'), ('county', county_number), ('block group', '*')]),
                             ['B02001_001E', 'B02001_003E', "B03001_001E", "B03001_003E", "B19013_001E", "B25118_002E", "B25118_014E", \
                             'B15003_002E', 'B15003_003E', 'B15003_004E', 'B15003_005E', 'B15003_006E', 'B15003_007E', \
                             'B15003_008E', 'B15003_009E', 'B15003_010E', 'B15003_011E', 'B15003_012E', 'B15003_013E', \
                             'B15003_014E', 'B15003_015E', 'B15003_016E', 'B15003_017E','B15003_018E', 'B15003_019E',\
                              'B15003_020E', 'B15003_021E', 'B15003_022E', 'B15003_023E','B15003_024E', 'B15003_025E'])
    # grabbing the Massachusetts data we want for each county
print(type(county_data["Suffolk County"]))
#B02001 is the race table: pulling black and total to form ratio
#B03001 is the hispanic table: pulling hispanic and total to form ratio
#B19013 is overall median income, Note that I could get black and white specific median incomes from B19013A and B
#B25118 is # of owner and renter occupied
#B15003 is all the education variables, going to be aggreagted later


Suffolk County
Barnstable County
Dukes County
Middlesex County
Berkshire County
Essex County
Bristol County
Hampden County
Plymouth County
Franklin County
Norfolk County
Hampshire County
Nantucket County
Worcester County
<class 'pandas.core.frame.DataFrame'>


In [7]:
lst_of_data = []
total = 0
for county in county_data:
    county_data[county]["county"] = county
    lst_of_data.append(county_data[county])
    total+=len(county_data[county])

final_ma = pd.concat(lst_of_data, axis=0)
final_ma.columns.values.tolist()

['B02001_001E',
 'B02001_003E',
 'B03001_001E',
 'B03001_003E',
 'B19013_001E',
 'B25118_002E',
 'B25118_014E',
 'B15003_002E',
 'B15003_003E',
 'B15003_004E',
 'B15003_005E',
 'B15003_006E',
 'B15003_007E',
 'B15003_008E',
 'B15003_009E',
 'B15003_010E',
 'B15003_011E',
 'B15003_012E',
 'B15003_013E',
 'B15003_014E',
 'B15003_015E',
 'B15003_016E',
 'B15003_017E',
 'B15003_018E',
 'B15003_019E',
 'B15003_020E',
 'B15003_021E',
 'B15003_022E',
 'B15003_023E',
 'B15003_024E',
 'B15003_025E',
 'county']

In [8]:
# clean educational attainment
final_ma['hs'] = final_ma[['B15003_002E', 'B15003_003E', 'B15003_004E', 'B15003_005E', 'B15003_006E', 'B15003_007E', \
                             'B15003_008E', 'B15003_009E', 'B15003_010E', 'B15003_011E', 'B15003_012E', 'B15003_013E', \
                             'B15003_014E', 'B15003_015E', 'B15003_016E', 'B15003_017E', 'B15003_018E']].sum(axis=1)
final_ma['somecollege'] = final_ma[['B15003_019E', 'B15003_020E', 'B15003_021E']].sum(axis=1)
final_ma['bach'] = final_ma[['B15003_022E', 'B15003_023E', 'B15003_024E', 'B15003_025E']].sum(axis=1)

In [9]:
# drop and rename
final_ma = final_ma.drop(columns=['B15003_002E', 'B15003_003E', 'B15003_004E', 'B15003_005E', 'B15003_006E', 'B15003_007E', \
                             'B15003_008E', 'B15003_009E', 'B15003_010E', 'B15003_011E', 'B15003_012E', 'B15003_013E', \
                             'B15003_014E', 'B15003_015E', 'B15003_016E', 'B15003_017E','B15003_018E', 'B15003_019E',\
                              'B15003_020E', 'B15003_021E', 'B15003_022E', 'B15003_023E','B15003_024E', 'B15003_025E'])
final_ma = final_ma.rename(columns={'B02001_001E': "total",'B02001_003E': "black", 'B03001_001E': "total", 'B03001_003E': "hispanic", 'B19013_001E': "median income",
 'B25118_002E': "owner occupied",'B25118_014E': "renter occupied"})
final_ma.columns.values.tolist()

['total',
 'black',
 'total',
 'hispanic',
 'median income',
 'owner occupied',
 'renter occupied',
 'county',
 'hs',
 'somecollege',
 'bach']

In [10]:
final_ma.to_csv("/Users/kmbrgandhi/Documents/Housing/Property_Assessments/propertyassessments/intermediate/checkthis.csv")